In [3]:
import pandas

data = pandas.read_csv('features.csv')
# data[['first_blood_time', 'first_blood_team']].fillna(-1, inplace=True)
data.drop(['match_id', 
        'barracks_status_dire',
        'tower_status_radiant',
        'tower_status_dire',
        'barracks_status_radiant',
        'duration'], inplace=True, axis=1)
data.fillna(0, inplace=True)
data = data.sample(frac=1).reset_index(drop=True)
y_train = data['radiant_win'].as_matrix()
x_train = data.drop(['radiant_win'], axis=1).as_matrix()

data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,radiant_win
0,1448094947,1,86,3,718,832,1,0,1,10,...,-32.0,161.0,-71.0,0.0,4,4,1,0,52.0,0
1,1446749067,1,51,4,1205,941,11,0,0,6,...,-35.0,0.0,-86.0,286.0,2,4,3,0,-11.0,0
2,1443159156,1,51,3,565,985,0,0,1,7,...,31.0,136.0,-70.0,284.0,2,4,3,1,-41.0,0
3,1449516916,1,21,5,2305,2173,21,2,0,12,...,-7.0,0.0,-72.0,0.0,7,3,2,0,-4.0,0
4,1449522438,1,68,3,1026,1547,6,3,0,13,...,18.0,0.0,-69.0,288.0,3,3,3,1,-2.0,0


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
data.columns[data.isnull().any()].tolist()


[]

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=250, random_state=241, learning_rate=0.1)
clf.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=250, presort='auto', random_state=241,
              subsample=1.0, verbose=0, warm_start=False)

In [9]:
data_test = pandas.read_csv('features_test.csv')
data_test.drop(['match_id'], inplace=True, axis=1)
data_test.fillna(0, inplace=True)
data_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
1,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,0.0,3,2,2,1,16.0
2,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
3,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
4,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [17]:
from sklearn.metrics import roc_auc_score

y_pred = clf.predict_proba(X_test)[:, 1]
roc_auc_score(y_true=y_test, y_score=y_pred)

0.7097214096969372

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

clf2 = LogisticRegression(C=0.1, random_state=241)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
# print(x_train_scaled)
clf2.fit(x_train_scaled, y_train)

roc_auc_score(y_true=y_test, y_score=clf2.predict_proba(scaler.transform(X_test))[:, 1])

0.70988396007163623